<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import streamlit as st
from streamlit_folium import folium_static
import folium
import wget
import pandas as pd

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [2]:
# Download and read the `spacex_launch_geo.csv`
#spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df= pd.read_csv('spacex_launch_geo.csv')

Now, you can take a look at what are the coordinates for each site.


In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [5]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker_jsc = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 15; color:darkred;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker_jsc)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [6]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Launch site coordinates and names
launch_sites = [
    {"name": "CCAFS LC-40", "lat": 28.562302, "lon": -80.577356},
    {"name": "CCAFS SLC-40", "lat": 28.563197, "lon": -80.576820},
    {"name": "KSC LC-39A", "lat": 28.573255, "lon": -80.646895},
    {"name": "VAFB SLC-4E", "lat": 34.632834, "lon": -120.610745}
]

# Create a red circle and marker for each launch site
for site in launch_sites:
    # Create a blue circle at the launch site's coordinate with a popup label showing its name
    circle = folium.Circle([site["lat"], site["lon"]], radius=1000, color='blue', fill=True).add_child(folium.Popup(site["name"]))
    site_map.add_child(circle)
    
    # Create a marker with custom icon at the launch site's coordinate
    marker = folium.Marker(
        [site["lat"], site["lon"]],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 15; color:darkred;"><b>%s</b></div>' % site["name"],
        )
    )
    site_map.add_child(marker)

# Display the map
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [7]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [8]:
# Create cluster object
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [9]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [10]:
# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
#for index, record in spacex_df.iterrows():
#    # TODO: Create and add a Marker cluster to the site map
#    # marker = folium.Marker(...)
#    marker_cluster.add_child(marker)

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# Iterate over each launch record
for index, row in spacex_df.iterrows():
    # Define marker color based on the class value
    marker_color = 'lightgreen' if row['class'] == 1 else 'darkred' # Light Green for successful launch and darkred for failed launch
    
    # Create a Marker object for the launch record
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=marker_color)
    )
    
    # Add the Marker object to the MarkerCluster
    marker.add_to(marker_cluster)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [11]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [ ]:
# Add the LatLngPopup feature
site_map.add_child(folium.LatLngPopup())

# Display the map in Streamlit
folium_static(site_map)

# Get the last clicked coordinates
#last_clicked = st.session_state.get('last_clicked')

#if last_clicked is not None:
#    st.write(f"You last clicked at lat: {last_clicked['lat']}, lon: {last_clicked['lon']}")

site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [12]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [13]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# Launch site coordinates and names

def calculate_distance(launch_site, coastline):
    lat1, lon1 = launch_site
    lat2, lon2 = coastline
    
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Launch sites with co-ordinates
CCAFS_LC_40 = (28.562302, -80.577356)
CCAFS_SLC_40 = (28.563197, -80.576820)
KSC_LC_39A = (28.573255, -80.646895)
VAFB_SLC_4E =  (34.632834, -120.610745)

# List launch sites
launch_list = [CCAFS_LC_40, CCAFS_SLC_40, KSC_LC_39A, VAFB_SLC_4E]

# Coastline co-ordinates
coastline_east = (28.56322, -80.56805)
coastline_west = (34.63335, -120.62607)


In [15]:
# Calculate the distance between the launch site and the closest coastline
# Calculate the distance between CCAFS_SLC_40 and coastline east
distance_to_coastline_east = calculate_distance(CCAFS_SLC_40, coastline_east)
print(f"Distance from CCAFS_SLC_40 to the closest coastline: {distance_to_coastline_east:.2f} km")


Distance from CCAFS_SLC_40 to the closest coastline: 0.86 km


In [ ]:
# Calculate distances
for site_coords, site_name in zip(launch_list, ["CCAFS_LC_40", "CCAFS_SLC_40", "KSC_LC_39A", "VAFB_SLC_4E"]):
    distance_to_east = calculate_distance(site_coords, coastline_east)
    distance_to_west = calculate_distance(site_coords, coastline_west)
    print(f"Launch site: {site_name}, Coordinates: {site_coords}, Distance to East Coastline: {distance_to_east:.2f} km, Distance to West Coastline: {distance_to_west:.2f} km")


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [19]:
# Add marker for the launch site closest to the east coastline
folium.Marker(
    location= CCAFS_SLC_40,
    popup=f"Distance to coastline: {distance_to_coastline_east:.2f} km",
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

# Display the map
site_map

In [ ]:
"""
# Add a marker for the launch site
folium.Marker(
    location=launch_site_cod,
    popup='Launch Site'
).add_to(site_map)

# Add a marker for the closest coastline
folium.Marker(
    location=coastline_cod,
    popup=f'Closest Coastline\nDistance: {distance_coastline:.2f} km',
    icon=folium.Icon(color='red')
).add_to(site_map)

# Display the map
site_map
"""

In [23]:
# Add marker for the east coastline with distance written in red
folium.Marker(
    location=coastline_east,
    popup=f"Coastline: {distance_to_coastline_east:.2f} km from CCAFS_SLC_40",
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 15; color:red;"><b>{distance_to_coastline_east:.2f} KM</b></div>'
    ),
    tooltip="East Coastline"
).add_to(site_map)

folium.Marker(
    location= coastline_east,
    popup=f"Distance from CCAFS_SLC_40: {distance_to_coastline_east:.2f} km",
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

# Display the map
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# Draw a PolyLine between launch site and closest coastline
lines = folium.PolyLine(locations=[launch_site_cod, coastline_cod], color='blue')

# Display the map
site_map.add_child(lines)
#site_map.add_child(dist_marker)
site_map


In [24]:
# Add lines connecting launch sites to coastlines
folium.PolyLine(
        locations=[CCAFS_SLC_40, coastline_east],
        color='blue',
        weight=2,
        popup=f"Distance: {distance_to_coastline_east:.2f} km"
    ).add_to(site_map)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [25]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Function to calculate distance between two points

# Cities
Titusville = (28.6114, -80.8076)
Santa_Maria = (34.9527, -120.4353)

# KSC_LC_39A is closest to Titusville
# Calculate the distance between the two points
distance_to_Titusville = calculate_distance(KSC_LC_39A, Titusville)

# VAFB_SLC_4E is closest to Santa Maria
# Calculate the distance between the two points
distance_to_Santa_Maria = calculate_distance(VAFB_SLC_4E, Santa_Maria)

# Create markers for the cities
folium.Marker(location=Titusville, popup=f'Titusville (Distance from KSC_LC_39A: {distance_to_Titusville:.2f} km)').add_to(site_map)
folium.Marker(location=Santa_Maria, popup=f'Santa Maria (Distance from VAFB_SLC_4E: {distance_to_Santa_Maria:.2f} km)').add_to(site_map)

# Create markers for the launch sites
folium.Marker(location=KSC_LC_39A, popup=f'KSC_LC_39A (Distance to Titusville: {distance_to_Titusville:.2f} km)').add_to(site_map)
folium.Marker(location=VAFB_SLC_4E, popup=f'VAFB_SLC_4E (Distance to Santa Maria: {distance_to_Santa_Maria:.2f} km)').add_to(site_map)

# Draw blue lines between each launch site and its corresponding city
folium.PolyLine(locations=[KSC_LC_39A, Titusville], color='blue').add_to(site_map)
folium.PolyLine(locations=[VAFB_SLC_4E, Santa_Maria], color='blue').add_to(site_map)

# Display the map
site_map


In [26]:
# Distances to railways and highways
# Add markers and draws blue lines between the launch sites and the railways and highways
# Railways and highways in the east and west regions
east_railway = (28.57116, -80.58542)
west_railway = (34.6339, -120.6247)
east_highway = (28.56344, -80.57082)
west_highway = (34.7069, -120.4854)

# CCAFS_SLC_40 is closest to east railway and east highway
# Calculate the distances from CCAFS_SLC_40 to east railway and east highway
distance_to_east_railway = calculate_distance(CCAFS_SLC_40, east_railway)
distance_to_east_highway = calculate_distance(CCAFS_SLC_40, east_highway)

# VAFB_SLC_4E is closest to west railway and west highway
# Calculate the distances from VAFB_SLC_4E to west railway and west highway
distance_to_west_railway = calculate_distance(VAFB_SLC_4E, west_railway)
distance_to_west_highway = calculate_distance(VAFB_SLC_4E, west_highway)

# Print the distances
print(f"Distance from CCAFS_SLC_40 to east railway: {distance_to_east_railway:.2f} km")
print(f"Distance from CCAFS_SLC_40 to east highway: {distance_to_east_highway:.2f} km")
print(f"Distance from VAFB_SLC_4E to west railway: {distance_to_west_railway:.2f} km")
print(f"Distance from VAFB_SLC_4E to west highway: {distance_to_west_highway:.2f} km")

Distance from CCAFS_SLC_40 to east railway: 1.22 km
Distance from CCAFS_SLC_40 to east highway: 0.59 km
Distance from VAFB_SLC_4E to west railway: 1.28 km
Distance from VAFB_SLC_4E to west highway: 14.12 km


In [27]:
# Markers and lines
# Create markers for the launch sites
folium.Marker(location=CCAFS_SLC_40, popup=f'CCAFS_SLC_40 (Distance to closest railway: {distance_to_east_railway:.2f} km, Distance to closest highway: {distance_to_east_highway:.2f} km)').add_to(site_map)
folium.Marker(location=VAFB_SLC_4E, popup=f'VAFB_SLC_4E (Distance to closest railway: {distance_to_west_railway:.2f} km, Distance to closest highway: {distance_to_west_highway:.2f} km)').add_to(site_map)

# Draw blue lines between launch sites and railways
folium.PolyLine(locations=[CCAFS_SLC_40, east_railway], color='blue').add_to(site_map)
folium.PolyLine(locations=[VAFB_SLC_4E, west_railway], color='blue').add_to(site_map)

# Draw blue lines between launch sites and highways
folium.PolyLine(locations=[CCAFS_SLC_40, east_highway], color='blue').add_to(site_map)
folium.PolyLine(locations=[VAFB_SLC_4E, west_highway], color='blue').add_to(site_map)

# Display the map
site_map

In [30]:
# Add markers for railways and highways with distances written in red
folium.Marker(
    location=east_railway,
    popup=f"Closest Railway: {distance_to_east_railway:.2f} km from CCAFS_SLC_40",
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 15; color:red;"><b>{distance_to_east_railway:.2f} KM</b></div>'
    ),
    tooltip="Closest Railway"
).add_to(site_map)

# Add markers for railways and highways with distances written in red
folium.Marker(
    location=east_railway,
    popup=f"Closest Railway: {distance_to_east_railway:.2f} km from CCAFS_SLC_40",
    icon=folium.Icon(color='red', icon='rocket')
).add_to(site_map)

folium.Marker(
    location=east_highway,
    popup=f"Closest Highway: {distance_to_east_highway:.2f} km from CCAFS_SLC_40",
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 15; color:red;"><b>{distance_to_east_highway:.2f} KM</b></div>'
    ),
    tooltip="Closest Highway"
).add_to(site_map)

folium.Marker(
    location=east_highway,
    popup=f"Closest Highway: {distance_to_east_highway:.2f} km from CCAFS_SLC_40",
    icon=folium.Icon(color='red', icon='rocket')
).add_to(site_map)

folium.Marker(
    location=west_railway,
    popup=f"Closest Railway: {distance_to_west_railway:.2f} km from VAFB_SLC_4E",
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 15; color:red;"><b>{distance_to_west_railway:.2f} KM</b></div>'
    ),
    tooltip="Closest Railway"
).add_to(site_map)

folium.Marker(
    location=west_railway,
    popup=f"Closest Railway: {distance_to_west_railway:.2f} km from VAFB_SLC_4E",
    icon=folium.Icon(color='red', icon='rocket')
).add_to(site_map)

folium.Marker(
    location=west_highway,
    popup=f"Closest Highway: {distance_to_west_highway:.2f} km from VAFB_SLC_4E",
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 15; color:red;"><b>{distance_to_west_highway:.2f} KM</b></div>'
    ),
    tooltip="Closest Highway"
).add_to(site_map)

folium.Marker(
    location=west_highway,
    popup=f"Closest Highway: {distance_to_west_highway:.2f} km from VAFB_SLC_4E",
    icon=folium.Icon(color='red', icon='rocket')
).add_to(site_map)


# Display the map
site_map




After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


**Answers**:
- Launch sites are in close proximity to railways
- Launch sites are not in close proximity to highways on the west coast but are close promxitmity to highways on the east coast
- Launch sites on both the east and the west coast are in close proximity to the coastline
- Launch sites appear to be at least 16 KM away from the closest cities on both the east coast and the west coast.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
